In [1]:
import pyodbc
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv("C:/Users/BoydClaire/.Renviron"))

def get_database_tables(database_name, connection_args, schema=None):

    connection_string = "".join(connection_args)

    # create connection object
    cnxn = pyodbc.connect(connection_string)

    cursor = cnxn.cursor()

    if schema:
        tables_raw = cursor.tables(schema=schema)
    else:
        tables_raw = cursor.tables()

    tables = pd.DataFrame(tables_raw)
    tables.columns = ['list']
    cleaned_tables = pd.DataFrame(tables['list'].apply(list).to_list(), columns = ['DATABASE_NAME','SCHEMA','NAME','TYPE','SELF_REFERENCING_COL_NAME'])
    cleaned_tables['DATABASE'] = database_name

    return cleaned_tables

True

In [32]:
# get sql tables

sql_database_list = ["production"]
sql_tables = []

for database in sql_database_list:

    # build connection string
    connection_args = ["Driver={SQL Server};",
                        f"SERVER={os.environ[f'{database}_server']};",
                        f"DATABASE={os.environ[f'{database}_database']};",
                        f"UID={os.environ[f'{database}_username']};",
                        f"PWD={os.environ[f'{database}_password']};",
                        f"Trusted_Connection=no;",
                        "Port=1433"
    ]

    table = get_database_tables(database, connection_args)
    sql_tables.append(table)

In [29]:
# get oracle tables

oracle_database_list = ["fdw", "ias"]
oracle_tables = []

for database in oracle_database_list:

    # build connection string
    database_schema = os.getenv(f"{database}_schema")
    connection_args = ["Driver={Oracle in OraClient19Home1};",
                        f"DBQ={os.environ[f'{database}_path']};"
                        f"DATABASE={os.environ[f'{database}_schema']};"
                        f"UID={os.environ[f'{database}_username']};",
                        f"PWD={os.environ[f'{database}_password']};",
                        f"Trusted_Connection=no;",
                        "Port=1433"
    ]

    table = get_database_tables(database, connection_args)
    table = table.loc[table.loc[:,"SCHEMA"] == database_schema.upper(), :]

    oracle_tables.append(table)

In [30]:
flat = [table for tables in [sql_tables, oracle_tables] for table in tables]
all_tables = pd.concat(flat, axis=0)

In [33]:
all_tables.groupby("DATABASE").count()

,DATABASE_NAME,SCHEMA,NAME,TYPE,SELF_REFERENCING_COL_NAME
DATABASE,,,,,
fdw,0,228,228,228,0
ias,0,6470,6470,6470,0
production,4684,4684,4684,4684,0


In [93]:
all_tables['DATABASE'] == "production"

0         True
1         True
2         True
3         True
4         True
         ...  
29820    False
29821    False
29822    False
29823    False
29824    False
Name: DATABASE, Length: 11382, dtype: bool

In [96]:
# filter tables 

display_tables = all_tables.loc[(~all_tables['NAME'].str.contains('_20[0-9]{2}')),:]

display_tables.groupby("DATABASE").count()

,DATABASE_NAME,SCHEMA,NAME,TYPE,SELF_REFERENCING_COL_NAME
DATABASE,,,,,
fdw,0,228,228,228,0
ias,0,6154,6154,6154,0
production,2407,2407,2407,2407,0


things to consider:
* how often records get updated -- last update

how to pull in cols for all databases based on the tables above
*

In [37]:
database= "test"

# build connection string
connection_args = ["Driver={SQL Server};",
                    f"SERVER={os.environ[f'{database}_server']};",
                    f"DATABASE={os.environ[f'{database}_database']};",
                    f"UID={os.environ[f'{database}_username']};",
                    f"PWD={os.environ[f'{database}_password']};",
                    f"Trusted_Connection=no;",
                    "Port=1433"
]

connection_string = "".join(connection_args)

# create connection object
cnxn = pyodbc.connect(connection_string)

cursor = cnxn.cursor()

In [74]:
def get_columns_from_table(cursor, tablename):
    
    cursor.execute(f"""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = '{tablename}'
    """)

    # Fetch all results
    columns_info = pd.DataFrame(cursor.fetchall())

    columns_info.columns = ['list']
    columns_info = pd.DataFrame(columns_info['list'].apply(list).to_list(), columns = ['COLUMN_NAME','TYPE'])

    return columns_info


def get_prefix_from_table(cursor, tablename):

    columns_info = get_columns_from_table(cursor, tablename)
    prefix = columns_info['COLUMN_NAME'].str.extract("^(.*?)_.*$")[0][0]
    
    return prefix

In [78]:
get_prefix_from_table(cursor, "BLDG")


'BLD'